In [1]:
# Step 1: Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.20.0


In [2]:
# Step 2: Dataset Path
dataset_dir = r'C:\Users\DELL\Downloads\kidneyy\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'



In [3]:

# Step 3: Image Data Generators (grayscale)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    color_mode='grayscale',      # ✅ grayscale
    class_mode='categorical',
    batch_size=32,
    subset='training',
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    color_mode='grayscale',      # ✅ grayscale
    class_mode='categorical',
    batch_size=32,
    subset='validation',
    seed=42
)

print("Class indices:", train_generator.class_indices)


Found 5164 images belonging to 2 classes.
Found 1290 images belonging to 2 classes.
Class indices: {'Normal': 0, 'Stone': 1}


In [4]:
# Step 4: Build Model (Custom CNN for Grayscale)
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,1)),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      22,151,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,244,610 (84.86 MB)

 Trainable params: 22,244,610 (84.86 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Step 5: Callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_accuracy"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3, verbose=1)
]


In [6]:
# Step 6: Train Model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=callbacks
)


C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7719 - loss: 0.5261  

162/162 ━━━━━━━━━━━━━━━━━━━━ 271s 2s/step - accuracy: 0.7930 - loss: 0.4564 - val_accuracy: 0.8481 - val_loss: 0.3688 - learning_rate: 0.0010
Epoch 2/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8469 - loss: 0.3609  

162/162 ━━━━━━━━━━━━━━━━━━━━ 240s 1s/step - accuracy: 0.8602 - loss: 0.3347 - val_accuracy: 0.8535 - val_loss: 0.3399 - learning_rate: 0.0010
Epoch 3/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8838 - loss: 0.2706  

162/162 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.8914 - loss: 0.2505 - val_accuracy: 0.8566 - val_loss: 0.3109 - learning_rate: 0.0010
Epoch 4/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8954 - loss: 0.2468  

...

162/162 ━━━━━━━━━━━━━━━━━━━━ 18219s 113s/step - accuracy: 0.9293 - loss: 0.1682 - val_accuracy: 0.9178 - val_loss: 0.2402 - learning_rate: 0.0010
Epoch 6/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - accuracy: 0.9423 - loss: 0.1436 - val_accuracy: 0.9101 - val_loss: 0.2354 - learning_rate: 0.0010
Epoch 7/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9592 - loss: 0.1201  

162/162 ━━━━━━━━━━━━━━━━━━━━ 218s 1s/step - accuracy: 0.9568 - loss: 0.1193 - val_accuracy: 0.9271 - val_loss: 0.2274 - learning_rate: 0.0010
Epoch 8/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - accuracy: 0.9646 - loss: 0.0929 - val_accuracy: 0.9062 - val_loss: 0.2696 - learning_rate: 0.0010
Epoch 9/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9622 - loss: 0.0977  

162/162 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - accuracy: 0.9721 - loss: 0.0788 - val_accuracy: 0.9403 - val_loss: 0.1809 - learning_rate: 0.0010
Epoch 10/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9821 - loss: 0.0579  

162/162 ━━━━━━━━━━━━━━━━━━━━ 219s 1s/step - accuracy: 0.9841 - loss: 0.0543 - val_accuracy: 0.9488 - val_loss: 0.1420 - learning_rate: 0.0010
Epoch 11/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.9845 - loss: 0.0519 - val_accuracy: 0.8519 - val_loss: 0.4389 - learning_rate: 0.0010
Epoch 12/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.9816 - loss: 0.0567 - val_accuracy: 0.9217 - val_loss: 0.1919 - learning_rate: 0.0010
Epoch 13/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9847 - loss: 0.0434  

162/162 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - accuracy: 0.9837 - loss: 0.0475 - val_accuracy: 0.9566 - val_loss: 0.1227 - learning_rate: 0.0010
Epoch 14/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.9901 - loss: 0.0306 - val_accuracy: 0.9186 - val_loss: 0.2804 - learning_rate: 0.0010
Epoch 15/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.9847 - loss: 0.0487 - val_accuracy: 0.9403 - val_loss: 0.2118 - learning_rate: 0.0010
Epoch 16/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9930 - loss: 0.0230  
Epoch 16: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
162/162 ━━━━━━━━━━━━━━━━━━━━ 218s 1s/step - accuracy: 0.9911 - loss: 0.0270 - val_accuracy: 0.9419 - val_loss: 0.2188 - learning_rate: 0.0010
Epoch 17/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 217s 1s/step - accuracy: 0.9967 - loss: 0.0122 - val_accuracy: 0.9364 - val_loss: 0.2618 - learning_rate: 3.0000e-04
Epoch 18/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.9975 - loss: 0.01

In [7]:
# Step 7: Evaluate
val_loss, val_acc = model.evaluate(validation_generator, verbose=2)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

41/41 - 38s - 924ms/step - accuracy: 0.9442 - loss: 0.1489
Validation Accuracy: 94.42%


In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to your test image
img_path = r"C:\Users\DELL\Downloads\kidneyy\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\Stone\Stone- (11).jpg"

# Load image in grayscale
img = image.load_img(img_path, target_size=(224,224), color_mode="grayscale")

# Convert to array
img_array = image.img_to_array(img)

# Normalize
img_array = img_array / 255.0

# Expand dims
img_array = np.expand_dims(img_array, axis=0)

# Predict
pred = model.predict(img_array)[0][0]  # ✅ scalar float now
print("Raw prediction (probability of Stone):", pred)

if pred >= 0.5:
    print("Predicted class: Normal")
else:
    print("Predicted class: Stone")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Raw prediction (probability of Stone): 0.06745462
Predicted class: Stone
